# Clustering With Howso Engine

## Overview

This example notebook will demonstrate Howso Engine’s ability to calculate pairwise distances for the training cases. This capability is based on Howso Engine’s underlying instance-based learning platform. Distances can be used for a variety of use cases, in particular, for clustering algorithms, such as HDBSCAN. Howso Engine’s feature reduction capabilities will also be used to demonstrate how to eliminate features for clustering. 

In [1]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
from sklearn.cluster import HDBSCAN
from sklearn.datasets import make_blobs
from sklearn.metrics import adjusted_rand_score

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_umap

### Step 1: Generate a simple dataset with noise

A 2-D blob is generated along with an extra noise column. The noise column is used to demonstrate feature reduction.


In [2]:
blobs = make_blobs(n_samples=100, n_features=2, centers=[(1,0), (1,5), (10,2)], random_state=0)
df = pd.DataFrame(blobs[0], columns=['x', 'y'])
df['target'] = blobs[1]

# Noise
noise = np.random.uniform(df['y'].min(), df['y'].max(), len(df))
df['noise'] = noise

### Step 2: Create the Trainee, Train, and Analyze

For questions about the specific steps of this section, please see the [basic workflow guide](https://docs.howso.com/en/release-latest/user_guide/basic_capabilities/basic_workflow.html).

In [3]:
# Infer feature attributes
features = infer_feature_attributes(df)

# Specify Context and Action Features
action_features = ['target']

context_features = features.get_names(without=action_features)

# Create the Trainee
t = engine.Trainee(features=features, overwrite_existing=True)

# Train
t.train(df)

# Targeted Analysis
t.analyze(context_features=context_features, action_features=action_features)

### Step 3: Inspect Feature Importance

In this example, since we have a target feature, we can inspect the [Accuracy Contributions](https://docs.howso.com/en/release-latest/getting_started/terminology.html#accuracy-contributions-ac) (a form of MDA) to understand how important each feature is.

In [4]:
results = t.react_aggregate(
    context_features=context_features,
    feature_influences_action_feature=action_features[0],
    details={
        "feature_robust_prediction_contributions": True,
        "feature_robust_accuracy_contributions": True
    }
)

In [5]:

# Robust Accuracy Contributions are returned as a matrix, the result must be edited to
# view the contributions of the target feature in a row as the other metrics.
robust_feature_influences = results['feature_robust_accuracy_contributions'].drop(['target'], axis=0)
robust_feature_influences

,target
noise,-0.044878
y,0.360162
x,0.268829


We can see that the noise column not only contributes less to the prediction than the other variables, but it also marginally increases (and perhaps lowers) the accuracy. This is expected from the noise column and demonstrates how unimportant features can be highlighted with Howso Engine.

### Step 4: Remove Unuseful Features

Unuseful features like the noise feature can reduce the quality of the clustering so we will remove them in this example. Howso Engine allows use to easily remove the features from the Trainee.

In [6]:
# Remove the noise feature
t.remove_feature('noise')

# update context_features
context_features.remove('noise')

# Reanalyze with the remaining features
t.analyze(context_features=context_features, action_features=action_features)

## Step 5: Get pairwise distances

By directly using Howso Engine's pairwise distances, we can take advantage of all the benefits of Howso Engine, including the ease of preprocessing and categorical variable handling. 

`get_distances` returns a square matrix of distances that make up the pairwise distances from each case to each other case in the model. The distances are computing using the Howso Engine's internal distance metric which has its parameters tuned within the Analyze call. For more information about the Howso Engine's distance metric, see [<em>Surprisal Driven k-NN for Robust and Interpretable Nonparametric Learning </em> (Banerjee et. al. 2023)](https://arxiv.org/abs/2311.10246).

In [7]:
# Pairwise distances can be obtained with a call to `get_distances`
x = t.get_distances()

x['distances']

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.000000,21.524230,0.327746,25.766165,26.799515,4.921362,1.067240,28.957057,16.969707,6.855511,...,3.690099,14.790771,16.188622,26.496615,1.171444,20.156596,29.935888,11.620102,16.028828,15.502804
1,21.524230,0.000000,21.200360,4.784261,3.727396,24.140167,18.337539,3.207865,35.104128,31.884971,...,14.097572,38.960900,38.350150,6.515157,16.590378,43.242837,5.506203,38.943793,35.680084,36.908997
2,0.327746,21.200360,0.000000,26.222720,27.261711,5.249224,1.374118,29.390801,17.473269,6.839916,...,3.637747,14.403318,15.980827,26.934698,0.712054,19.934056,30.357207,12.277769,16.563216,16.087465
3,25.766165,4.784261,26.222720,0.000000,0.983577,18.368802,28.565505,2.174458,27.619123,19.510849,...,25.326712,31.337838,30.249602,1.548202,28.070979,35.939560,2.159704,31.556218,28.165839,29.497095
4,26.799515,3.727396,27.261711,0.983577,0.000000,19.257272,29.680496,1.528669,29.031359,19.891137,...,26.482427,32.707983,31.571004,2.779735,29.160705,37.332879,2.975005,32.959152,29.569941,30.908541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20.156596,43.242837,19.934056,35.939560,37.332879,22.608402,21.698184,34.922993,6.816637,14.614922,...,26.045002,3.498635,2.976214,33.993665,20.747820,0.000000,32.948772,3.618099,6.352014,5.306551
96,29.935888,5.506203,30.357207,2.159704,2.975005,23.462080,32.356459,1.416218,24.338398,27.137586,...,29.056265,27.990339,26.367963,2.508197,31.961899,32.948772,0.000000,28.440964,24.866061,26.327341
97,11.620102,38.943793,12.277769,31.556218,32.959152,17.378717,13.204901,30.481279,3.198514,17.738374,...,20.586191,1.563087,3.957025,29.573717,13.315332,3.618099,28.440964,0.000000,2.729270,1.686107
98,16.028828,35.680084,16.563216,28.165839,29.569941,12.977910,17.372060,26.998426,0.408343,18.750655,...,15.894025,2.501151,3.997521,26.138063,17.578908,6.352014,24.866061,2.729270,0.000000,0.999975


## Step 6a: Cluster

Having access to the full matrix of pairwise distances, users have the ability to cluster with whatever algorithm they prefer (provided it supports a matrix of precomputed pairwise distances as an input).

In the case of this recipe, we demonstrate using the HDBSCAN provided through Scikit-Learn.

In [8]:
hdb = HDBSCAN(min_cluster_size=15, max_cluster_size=50, min_samples=10, metric='precomputed').fit(x['distances'])
hdbscan_labels = hdb.labels_
df["hdbscan_labels"] = hdbscan_labels

fig = make_subplots(rows=1, cols=2, subplot_titles=["True Clusters", "HDBSCAN Clusters"])
for label, group in df.groupby("target"):
    fig.add_scatter(x=group.x, y=group.y, col=1, row=1, mode="markers", name=label)
for label, group in df.groupby("hdbscan_labels"):
    fig.add_scatter(
        x=group.x, y=group.y, col=2, row=1, mode="markers", name=label,
        marker_color=fig.layout["template"]["layout"]["colorway"][label],
        showlegend=False
    )
fig.show()

## Step 6b: Visualize with UMAP

UMAP can be used in tandem with Howso Engine to visualize high-dimensional datasets, similar to how HDBSCAN is used.

In [9]:
fig = plot_umap(t, color="target", action_feature=action_features[0])
fig.show()

## Step 7: Inspect Accuracy 

We calculate the performance of the clustering by measuring the similarity of the clusters compared to the original labels.

In [10]:
dist_contribution_acc = adjusted_rand_score(df['target'], hdbscan_labels)
print(f'clustering accuracy score: {dist_contribution_acc}')

clustering accuracy score: 1.0


# Conclusion

As we demonstrate here, it is quite simple to use the Howso Engine for clustering using the internal distance metric by retrieving the pairwise distance matrix with `get_distances`. This pairwise distance metric can be passed to many popular clustering algorithms.